Code to make an SQLite database from ABCD data download

https://www.sqlitetutorial.net/sqlite-python/creating-tables/

In [19]:
import sqlite3
import os 
from sqlite3 import Error
from pathlib import Path
import pandas as pd
import numpy as np
import logging

In [11]:
os.chdir(r"/Users/margotwagner/ucsd/research/ABCD")

In [2]:
# logging
# Chang according to desired logging level (default it warning)
# logging.basicConfig(level=logging.INFO)

In [3]:
def create_connection(db_file):
    """ create a database connection to a SQLite database 
    params:
        db_file: 
            database file name to be connected (no .txt/.csv ending)

    return:
        connection object or None
    """
    
    # start database as empty text file
    Path(db_file).touch()


    # create sqlite table
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        logging.info("connection established")
        logging.info("sqlite version: {}".format(sqlite3.version))
    except Error as e:
        logging.error("Unable to establish connection")
        logging.error(e, exc_info=True)

    return conn

In [66]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    params:
        conn: 
            Connection object
        create_table_sql: 
            a CREATE TABLE statement

    returns:
    """
    success = 0

    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        logging.error("Unable to create table")
        logging.error(e, exc_info=True)

    return success

In [8]:
def get_txt_data(file_name):
    """ creates pandas dataframe containing data values from .txt data files
    
    params:
        file_name:
            title of file of interest (no .txt ending)

    return:
        table: 
            pandas dataframe containing data
        col_desc:
            np array containing column descriptions as given in .txt file
    
    """

    # Create table from txt file
    table = pd.read_csv('./data/data-files/{}.txt'.format(file_name), delimiter='\t')
    col_desc = table.loc[0].values  # get column descriptions
    table.drop(labels=0, inplace=True)    # remove column descriptions

    return table, col_desc


In [57]:
def get_csv_info(file_name):
    """ creates pandas dataframe containing dataset information/descriptions from csv files 
    
    params:
        file_name:
            title of file of interest (no .csv ending)

    return:
        pandas dataframe containing data information
    
    """

    # Create table from csv file
    table = pd.read_csv('./data/abcd-4.0-data-dictionaries/{}.csv'.format(file_name))

    # convert to sql data types
    table.replace({'GUID': 'TEXT', 'String': 'TEXT', 'Date': 'TEXT', 'Integer': 'INTEGER', 'Float': 'REAL'}, inplace=True)    # replace DataType with SQLlite data types

    return table

In [58]:
def dtypes_dict_maker(file_name):
    """ creates datatypes dict for all element names using info from csv files 
    
    params:
        file_name:
            title of file of interest (no .csv ending)

    return:
        dict containing datatype information
    
    """
    # get info table
    table = get_csv_info(file_name)

    data_types = table.to_dict()['DataType']

    # add common element names
    common_elems = ['collection_id', 'abcd_bp01_id', '{}'.format(file_name), 'collection_title', 'study_cohort_name']
    for e in common_elems:
        data_types[e] = 'TEXT'

    return data_types
    

In [70]:
def elem_desc_dict_maker(file_name):
    """ creates description dict for all elements using txt files
    
    params:
        file_name:
            title of file of interest (no .txt ending)

    return:
        dict containing element description information
    
    """

    table, col_desc = get_txt_data(file_name)

    # replace bad descriptions
    poor_desc = ['collection_id', '{}_id'.format(file_name), 'dataset_id', 'collection_title', 'study_cohort_name']
    good_desc = ['The collection of origin for the record', 'The globally unique row ID of the record in the NDA database', 'The record’s dataset of origin', 'The name of the collection', 'The name of the study cohort the data is from (data release)']
    
    for i in range(len(poor_desc)):
        # replace descriptions
        col_desc = np.where(col_desc == poor_desc[i], good_desc[i], col_desc)

    elem_keys = table.columns.values

    desc_dict = { elem_keys[i] : col_desc[i] for i in range(len(elem_keys)) }
    
    return desc_dict

In [41]:
def new_col_entry(column_name, data_type, comment='', constraint='', last_elem=False):
    """ creates entry for sqlite table 
    Note: sqlite python API req triple quotes for command to be read
    
    params:

    return:
        str with entry to put in CREATE TABLE command 
    
    """
    entry = "\t\t\t{} {} {}, \t -- {}\n"

    # strip new lines
    column_name = column_name.replace("\n", " ")
    data_type = data_type.replace("\n", " ")
    comment = comment.replace("\n", " ")

    # no comma for last element
    if last_elem:
        entry = "\t\t\t{} {} {} \t -- {}\n"

    return entry.format(column_name, data_type, constraint, comment)

In [64]:
def sql_create_table_cmd_builder(file_name):
    """ constructs CREATE TABLE entry for sqlite table using abcd file
    Note: sqlite python API req triple quotes for command to be read
    Note: could update to just use pandas to_sql function (maintain data types part of function)
    
    params:
        file_name:

    return:
        str with sqlite CREATE TABLE command
    """

    # open CREATE TABLE command
    #sql_cmd = ''' CREATE TABLE IF NOT EXISTS {} (
	#		'''.format(file_name)
    sql_cmd = ''' DROP TABLE IF EXISTS {}; CREATE TABLE {} (
			'''.format(file_name, file_name)

    
    # Manually add in first 3 ids from txt file (common to all .txts)        
    sql_cmd += new_col_entry("collection_id", "TEXT", "Record's collection of origin")        
    sql_cmd += new_col_entry("{}_id".format(file_name), "TEXT", "The globally unique row ID of the record in the NDA database")
    sql_cmd += new_col_entry("dataset_id", "TEXT", "The record's dataset of origin", last_elem=True)

    # Get column info
    info_table = get_csv_info(file_name)

    # Add remaining data column entries
    for index, row in info_table.iterrows():
        if row['ElementName'] == 'subjectkey':
            sql_cmd += new_col_entry(row['ElementName'], row['DataType'], row['ElementDescription'], constraint='PRIMARY KEY')

        elif index == len(info_table.index) - 1:
            sql_cmd += new_col_entry(row['ElementName'], row['DataType'], row['ElementDescription'], last_elem=True)

        else:
            sql_cmd += new_col_entry(row['ElementName'], row['DataType'], row['ElementDescription'])
            
    # close CREATE TABLE command
    sql_cmd += """);"""

    return sql_cmd

In [93]:
def elem_desc_df_maker(file_name):
    """ creates a description df for elements using txt files
    
    params:
        file_name:
            title of file of interest (no .txt ending)

    return:
        df containing element description information
    
    """

    desc_dict = elem_desc_dict_maker(file_name)

    desc_df = pd.DataFrame.from_dict(desc_dict, orient='index', columns=['elem_desc'])

    # move index to elements
    desc_df.reset_index(inplace=True)
    desc_df = desc_df.rename(columns = {'index':'elem_name'})

    return desc_df


elem_desc_df_maker('abcd_bp01')

,elem_name,elem_desc
0,collection_id,The collection of origin for the record
1,abcd_bp01_id,The globally unique row ID of the record in th...
2,dataset_id,The record’s dataset of origin
3,subjectkey,The NDAR Global Unique Identifier (GUID) for r...
4,src_subject_id,Subject ID how it's defined in lab/project
5,interview_date,Date on which the interview/genetic test/sampl...
6,interview_age,Age in months at the time of the interview/tes...
7,sex,Sex of subject at birth
8,eventname,The event name for which the data was collected
9,blood_pressure_start_time,Start time


In [95]:
def get_file_list(main_dir):
    """ Get list of data files that have the data and info

    param:
        dir:
            data directory to look in
    
    return
        file_names:
            list of file names with both data and documentation


    """
    # build directories
    data_dir = os.path.join(main_dir, 'data/data-files')
    docs_dir = os.path.join(main_dir, 'data/abcd-4.0-data-dictionaries')

    # Get list of all data file name
    data_file_names = [f.split('.')[0] for f in os.listdir(data_dir)]
    docs_file_names = [f.split('.')[0] for f in os.listdir(docs_dir)]

    # Check that data files all have info files
    only_data_files = sorted(list(set(data_file_names) - set(docs_file_names)))   # file names with txt data but no csv info
    only_info_files = sorted(list(set(docs_file_names) - set(data_file_names)))   # file names with csv info but no txt data  

    # Only want data files associated info file (for now)
    file_names = data_file_names.copy()

    if only_data_files != []:
        logging.warning("Ignoring data files without info files. Data file names ignore: \t")

        # don't create tables for data files without info 
        for f in only_data_files:
            file_names.remove(f)
            logging.warning("{}.txt".format(f))

        # TODO: ALSO GET FILES WITH JUST DATA BUT GET NAMES AND DESC FROM DATA FILE 

    logging.info("Files with info and no data:", only_info_files)

    return file_names

In [106]:
def main():
    """ Create ABCD sqlite3 database and populate with all given data tables.

    Note: current implementation uses pandas' df.to_sql function which does not allow for specification of primary key or commenting

    TODO: Add SQL command to add primary key and comments
    """

    # Change to ABCD directory
    main_dir = r"/Users/margotwagner/ucsd/research/ABCD"
    os.chdir(main_dir)

    # Name database
    database = r"./data/abcd.db"
    logging.info("Creating database", database)

    # Get files list
    file_names = get_file_list(main_dir)
    n_files = len(file_names)
    tables_created = []

    # create table for all data files
    print("beginning now...")
    logging.info("Attempting to create", n_files, "tables")
    success_count = 0
    fail_count = 0

    for f in file_names:

        # populate sql table with data
        data_table, col_desc = get_txt_data(f)

        # create a database connection
        conn = create_connection(database)

        # get datatypes
        dtypes_dict = dtypes_dict_maker(f)

        # create sql table in database 
        try:
            data_table.to_sql(f, con=conn, dtype=dtypes_dict, index=False)
            logging.info("Table {} created and populated successfully".format(f))
            success_count += 1 
            tables_created.append(f)

            # append elements in file and their descriptions to description table
            desc_df = elem_desc_df_maker(f)
            desc_df.to_sql('all_element_descriptions', con=conn, if_exists='append', index=False)

        except ValueError as ve:
            logging.error("Unable to create table for {} as it already exists".format(f))
            print("Data table for {} was not created".format(f))
            logging.error(ve, exc_info=True)
            fail_count += 1       

        # Output message
        if ((success_count + fail_count)%10 == 0):
            print("{:.1%} completion with {} tables successfully completed and {} failures".format(((success_count+fail_count)/n_files), success_count, fail_count))
        
    logging.info("Percent of tables successfully created: {:.0%}".format(success_count/n_files))
    logging.info("Percent of files that were not created: {:.0%}".format(fail_count/n_files))

    logging.info("Completed!")

    print("Finished!")

    return tables_created
    

if __name__ == '__main__':
    tables_created = main()     

beginning now...


/Users/margotwagner/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,5,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,24

2.7% completion with 10 tables successfully completed and 0 failures


/Users/margotwagner/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,5,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,24

5.4% completion with 20 tables successfully completed and 0 failures


...

94.3% completion with 350 tables successfully completed and 0 failures


/Users/margotwagner/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,6,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/wp/3f0wwp4n2yzg421rx1jgph980000gn/T/ipykernel_7573/18853167.py:47: DtypeWarning: Columns (0,1,2,6,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80) have mixed types.Specify dtype option on import or set low_memory=False.
  desc_df = elem_desc_df_maker(f)
/Users/margotwagner/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshe

97.0% completion with 360 tables successfully completed and 0 failures


/Users/margotwagner/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,6,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/wp/3f0wwp4n2yzg421rx1jgph980000gn/T/ipykernel_7573/18853167.py:47: DtypeWarning: Columns (0,1,2,6,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  desc_df = elem_desc_df_maker(f)
/Users/margotwagner/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,6,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77

99.7% completion with 370 tables successfully completed and 0 failures
Finished!


In [ ]:
def main():
    # Change to ABCD directory
    os.chdir(r"/Users/margotwagner/ucsd/research/ABCD")
    
    # Name database
    database = r"./data/abcd.db"
    logging.info("Creating database", database)

    # Get list of all data file name
    data_file_names = [f.split('.')[0] for f in os.listdir('./data/data-files')]
    info_file_names = [f.split('.')[0] for f in os.listdir('./data/abcd-4.0-data-dictionaries')]

    # Check that data files all have info files
    only_data_files = sorted(list(set(data_file_names) - set(info_file_names)))   # file names with txt data but no csv info
    only_info_files = sorted(list(set(info_file_names) - set(data_file_names)))   # file names with csv info but no txt data  

    # Only want data files associated info file (for now)
    file_names = data_file_names.copy()

    if only_data_files != []:
        logging.warning("Ignoring data files without info files. Data file names ignore: \t")

        # don't create tables for data files without info 
        for f in only_data_files:
            file_names.remove(f)
            logging.warning("{}.txt".format(f))

        # TODO: ALSO GET FILES WITH JUST DATA BUT GET NAMES AND DESC FROM DATA FILE 

    logging.info("Files with info and no data:", only_info_files)
    
    # create table for all data files
    # for f in file_names:
    for f in ['abcd_bp01']:

        # get file name from list
        logging.info("Creating table for file:", f)
    
        # sqlite statements
        sql_cmd = sql_create_table_cmd_builder(f)
        logging.info("Using the following sqlite command: \n", sql_cmd)

        # create a database connection
        conn = create_connection(database)

        # create tables
        if conn is not None:
            # create table
            table_created = create_table(conn, sql_cmd)

            if table_created:
                # populate sql table with data
                data_table, col_desc = get_txt_data(f)

                logging.info("Table created and populated successfully")
        

if __name__ == '__main__':
    main()
